# Diving into Deep Learning

In this assignment, we will dive right using machine learning to model how bad climate change will be. 

We will use the climate bench dataset, whose processing is covered in the example notebook. 

In [19]:
import xarray as xr

In [22]:
import torch
import xarray as xr
import numpy as np
from torch import nn
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt
import pandas as pd

In [25]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


### Loading Data into Batches
[Loading Data in Pytorch](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html)

In [51]:
from torch.utils.data import DataLoader, Dataset

In [27]:
X_train = xr.open_dataset("data_input_train.nc")

In [29]:
X_train

<xarray.Dataset>
Dimensions:  (time: 753)
Coordinates:
  * time     (time) int64 0 1 2 3 4 5 6 7 8 ... 745 746 747 748 749 750 751 752
Data variables:
    CO2      (time) float64 ...
    CH4      (time) float64 ...

In [30]:
Y_train = xr.open_dataset("data_output_train.nc")
Y_train

<xarray.Dataset>
Dimensions:    (latitude: 96, longitude: 144, time: 753)
Coordinates:
  * latitude   (latitude) float64 -90.0 -88.11 -86.21 ... 86.21 88.11 90.0
  * longitude  (longitude) float64 0.0 2.5 5.0 7.5 ... 350.0 352.5 355.0 357.5
  * time       (time) int64 0 1 2 3 4 5 6 7 ... 745 746 747 748 749 750 751 752
Data variables:
    tas        (time, latitude, longitude) float32 ...

In [38]:
Y = Y_train.mean(dim=['latitude', 'longitude'])
Y

<xarray.Dataset>
Dimensions:  (time: 753)
Coordinates:
  * time     (time) int64 0 1 2 3 4 5 6 7 8 ... 745 746 747 748 749 750 751 752
Data variables:
    tas      (time) float32 0.09651 0.03811 0.1706 0.2011 ... 1.144 1.129 1.185

In [54]:
co2 = X_train["CO2"]
ch4 = X_train["CH4"]

In [56]:
co2

<xarray.DataArray 'CO2' (time: 753)>
array([1.882972e-01, 3.772445e-01, 5.738141e-01, ..., 1.429420e+03,
       1.464818e+03, 1.500436e+03])
Coordinates:
  * time     (time) int64 0 1 2 3 4 5 6 7 8 ... 745 746 747 748 749 750 751 752

In [42]:
co2.data[3]

0.7788484984790588

In [49]:
tas = Y.tas.data
torch.tensor([tas[6]], 

0.16564173

In [57]:
class MyDataset(Dataset):
    def __init__(self, co2, ch4, tas):
        self.carbon = co2
        self.methane = ch4
        self.temp = tas

    def __len__(self):
        return(self.carbon.shape[0])

    def __getitem__(self, index):
        X = torch.tensor([self.carbon.data[index], self.methane.data[index]], dtype=torch.float32)
        y = torch.tensor([self.temp[index]], dtype=torch.float32)
        return(X, y)

In [63]:
train = MyDataset(co2, ch4, tas)
train_dataloader = DataLoader(train,16, shuffle=True)

In [79]:
train[0]

(tensor([0.1883, 0.0313]), tensor([0.0965]))

In [6]:
trainingDataset = 0

testDataset = 0

### Define Model Architecture

Helpful Links:
- [Neural Network Class](https://pytorch.org/docs/stable/generated/torch.nn.Module.html)
- [Torch Quickstart](https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html)


In [77]:
class MyNeuralNetwork(nn.Module):
    def __init__(self, input_size, output_size=1, num_layers=2, layer_size=32):
        super().__init__()
        self.relu = nn.ReLU()
        self.network = nn.Sequential(
            nn.Linear(in_features=input_size, out_features=layer_size),
            self.relu,
        )
        for layer in range(num_layers):
            next_layer = nn.Linear(in_features=layer_size, out_features=layer_size)
            self.network.append(next_layer)
            self.network.append(self.relu)
        
        output_layer = nn.Linear(in_features=layer_size, out_features=output_size)
        self.network.append(output_layer)
    def forward(self, X):
        return(self.network(X))

In [89]:
model = MyNeuralNetwork(input_size=2, output_size=1, num_layers=1, layer_size=32)
model

MyNeuralNetwork(
  (relu): ReLU()
  (network): Sequential(
    (0): Linear(in_features=2, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=32, bias=True)
    (3): ReLU()
    (4): Linear(in_features=32, out_features=1, bias=True)
  )
)

## Run Training Loop

Helpful Resources:
- [Optimizer Object](https://pytorch.org/docs/stable/optim.html)
- 


In [95]:
import torch.optim as optim

In [93]:
# set hyperparameters
n_neuron       = 16
num_epochs     = 5
learning_rate  = 0.001
minibatch_size = 64
model_num      = 1
N_layers       = 2 # number of hidden layers

In [102]:
loss_function = nn.MSELoss()
loss_function

MSELoss()

In [10]:
model = MyNeuralNetwork().to(device)
print(model)

MyNeuralNetwork()


In [92]:
#for param in model.parameters():
    #print(param)

In [96]:
optim = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)

In [98]:
a, b = train[7]

In [99]:
a

tensor([1.7886, 0.0344])

In [ ]:
for epoch in range(num_epochs):
    for (X,y) in train_dataloader:
        prediction = model(X)
        loss = loss_function(prediction, y)
        optim.step()
        optim.zero_grad
        

### Saving and Evaluating Model